# NoETL Execution Validation Notebook

Validate a playbook execution by inspecting server APIs, database tables, and local logs.

- Uses HTTP API to fetch execution summary and events
- Queries Postgres "noetl.error_log" for recent errors
- Reads final table "weather_alert_summary" for inserted rows
- Optionally inspects local JSON logs in `logs/`


In [19]:
# Configuration
import os, json, time, pathlib

# Server
HOST = os.environ.get('NOETL_HOST', 'localhost')
PORT = int(os.environ.get('NOETL_PORT', '8082'))
BASE = f'http://{HOST}:{PORT}/api'

# Database (defaults align with .env)
PGHOST = os.environ.get('POSTGRES_HOST', os.environ.get('PGHOST', 'localhost'))
PGPORT = int(os.environ.get('POSTGRES_PORT', os.environ.get('PGPORT', '30543')))
PGUSER = os.environ.get('POSTGRES_USER', os.environ.get('PGUSER', 'demo'))
PGPASSWORD = os.environ.get('POSTGRES_PASSWORD', os.environ.get('PGPASSWORD', 'demo'))
PGDATABASE = os.environ.get('POSTGRES_DB', os.environ.get('PGDATABASE', 'demo_noetl'))

# If you already know the execution id, set it here (as string or int)
EXECUTION_ID = os.environ.get('NOETL_LAST_EXECUTION_ID') or ''

# Resolve logs directory robustly when running from notebooks/
LOGS_DIR_ENV = os.environ.get('NOETL_LOG_DIR')
LOGS_DIR_CANDIDATES = []
if LOGS_DIR_ENV: LOGS_DIR_CANDIDATES.append(pathlib.Path(LOGS_DIR_ENV))
LOGS_DIR_CANDIDATES += [pathlib.Path('logs'), pathlib.Path('../logs'), pathlib.Path('../../logs')]
LOGS_DIR = next((p for p in LOGS_DIR_CANDIDATES if p.exists()), pathlib.Path('logs'))

print('Server:', BASE)
print('DB:', PGHOST, PGPORT, PGUSER, PGDATABASE)
print('Logs dir:', str(LOGS_DIR), 'exists:', LOGS_DIR.exists())


Server: http://localhost:8082/api
DB: localhost 30543 demo demo_noetl
Logs dir: ../logs exists: True


In [20]:
# Helper HTTP GET with stdlib fallback
def http_get_json(url: str):
    try:
        import requests  # type: ignore
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        return r.json()
    except Exception:
        import urllib.request, urllib.error
        try:
            with urllib.request.urlopen(url, timeout=30) as resp:
                return json.loads(resp.read().decode('utf-8'))
        except Exception as e:
            print('HTTP error:', e)
            return None

def pretty(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))


In [21]:
# Try to infer EXECUTION_ID if not provided
if not EXECUTION_ID:
    # 1) Try status.json in repo root
    st = pathlib.Path('status.json')
    if st.exists():
        try:
            data = json.loads(st.read_text(encoding='utf-8'))
            eid = data.get('id') or data.get('execution_id')
            if eid:
                EXECUTION_ID = str(eid)
        except Exception:
            pass

# 2) Try server /executions to pick the most recent
if not EXECUTION_ID:
    ex_list = http_get_json(f'{BASE}/executions')
    if isinstance(ex_list, list) and ex_list:
        # Items are dicts; keep first
        eid = ex_list[0].get('execution_id') or ex_list[0].get('id')
        if eid:
            EXECUTION_ID = str(eid)

print('EXECUTION_ID =', EXECUTION_ID or '(set me)')


EXECUTION_ID = 222437731135913984


In [22]:
# Fetch execution summary
if EXECUTION_ID:
    summary = http_get_json(f'{BASE}/executions/{EXECUTION_ID}')
    print('Execution summary:')
    pretty(summary)
else:
    print('Please set EXECUTION_ID above.')


Execution summary:
{
  "id": "222437731135913984",
  "playbook_id": "",
  "playbook_name": "Unknown",
  "status": "completed",
  "start_time": "2025-09-05T14:28:07.806437",
  "end_time": "2025-09-05T14:28:10.777129",
  "duration": 2.970692,
  "progress": 100,
  "result": {},
  "error": null,
  "events": [
    {
      "event_id": "222437731148496897",
      "event_type": "execution_start",
      "node_id": "222437731148496897",
      "node_name": "city_process",
      "node_type": "playbook",
      "status": "in_progress",
      "duration": 0.0,
      "timestamp": "2025-09-05T14:28:07.806437",
      "input_context": {
        "path": "examples/weather/city_process",
        "version": "0.1.0",
        "workload": {
          "city": "{'lat': 48.85, 'lon': 2.35, 'name': 'Paris'}",
          "base_url": "",
          "temperature_threshold": ""
        }
      },
      "output_result": {},
      "metadata": {
        "playbook_path": "examples/weather/city_process",
        "resource_path

In [23]:
# Fetch all events for the execution
events = None
if EXECUTION_ID:
    events = http_get_json(f'{BASE}/events/by-execution/{EXECUTION_ID}')
    print('Event count:', len(events.get('events', [])) if isinstance(events, dict) else 'n/a')
    # Show last 10 event summaries
    if events and isinstance(events, dict):
        for e in (events.get('events') or [])[-10:]:
            print(e.get('timestamp'), e.get('node_name'), e.get('event_type'), e.get('status'))
else:
    print('No events (or EXECUTION_ID not set).')


Event count: 17
2025-09-05T14:28:10.235032 evaluate_weather action_started RUNNING
2025-09-05T14:28:10.244789 evaluate_weather action_completed COMPLETED
2025-09-05T14:28:10.274040 evaluate_weather action_completed COMPLETED
2025-09-05T14:28:10.540106 alert_step action_started RUNNING
2025-09-05T14:28:10.553019 alert_step action_completed COMPLETED
2025-09-05T14:28:10.707946 log_step action_started RUNNING
2025-09-05T14:28:10.724445 log_step action_completed COMPLETED
2025-09-05T14:28:10.744885 end action_completed COMPLETED
2025-09-05T14:28:10.776809 examples/weather/city_process execution_complete completed
2025-09-05T14:28:10.777129 examples/weather/city_process execution_complete completed


In [24]:
# Quick inline validation:
# - Count loop iterations for city_loop
# - Ensure at least one COMPLETED action exists
if events and isinstance(events, dict):
    ev = events.get('events') or []
    loop_iters = [e for e in ev if e.get('event_type') == 'loop_iteration' and e.get('node_name') == 'city_loop']
    completed = [e for e in ev if (e.get('status') or '').lower() in ('completed','success')]
    print('city_loop.iterations =', len(loop_iters))
    print('completed events =', len(completed))
else:
    print('Events not loaded.')


city_loop.iterations = 0
completed events = 10


In [25]:
# Inspect Postgres error log (noetl.error_log)
try:
    import psycopg
    conn = psycopg.connect(dbname=PGDATABASE, user=PGUSER, password=PGPASSWORD, host=PGHOST, port=PGPORT)
    with conn.cursor() as cur:
        cur.execute(
            """
            SELECT error_id, error_type, left(error_message, 160) AS msg, to_char(timestamp,'YYYY-MM-DD HH24:MI:SS') AS ts
            FROM noetl.error_log
            ORDER BY timestamp DESC
            LIMIT 10
            """
        )
        rows = cur.fetchall()
        print('Recent errors:')
        for r in rows:
            print(r)
    conn.close()
except Exception as e:
    print('Error querying Postgres error_log:', e)


Recent errors:


In [26]:
# Inspect final table (weather_alert_summary)
try:
    import psycopg
    conn = psycopg.connect(dbname=PGDATABASE, user=PGUSER, password=PGPASSWORD, host=PGHOST, port=PGPORT)
    with conn.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema='public' AND table_name='weather_alert_summary'")
        exists = (cur.fetchone() or [0])[0] == 1
        if exists:
            cur.execute("SELECT id, alert_cities, alert_count, to_char(created_at,'YYYY-MM-DD HH24:MI:SS') FROM weather_alert_summary ORDER BY id DESC LIMIT 10")
            print('weather_alert_summary (last 10):')
            for row in cur.fetchall():
                print(row)
        else:
            print('weather_alert_summary table does not exist yet.')
    conn.close()
except Exception as e:
    print('Error querying weather_alert_summary:', e)


weather_alert_summary (last 10):


In [27]:
# Optional: inspect local JSON logs if present
for name in ('event_log.json', 'queue.json', 'runtime.json'):
    p = LOGS_DIR / name
    if p.exists():
        try:
            # Read a small prefix for sanity
            text = p.read_text(encoding='utf-8')
            print(f'--- {p} (first 600 chars) ---')
            print(text[:600])
        except Exception as e:
            print(f'Failed to read {p}:', e)
    else:
        print(f'{p} not found.')


--- ../logs/event_log.json (first 600 chars) ---
[
  {
    "execution_id": 222437726840946688,
    "event_id": 222437726857723904,
    "parent_event_id": null,
    "timestamp": "2025-09-05T14:28:06.776403",
    "event_type": "execution_start",
    "node_id": "222437726857723904",
    "node_name": "weather_loop_example",
    "node_type": "playbook",
    "status": "in_progress",
    "duration": 0,
    "input_context": "{\"path\": \"examples/weather/weather_loop_example\", \"version\": \"0.1.0\", \"workload\": {\"jobId\": \"{{ job.uuid }}\", \"state\": \"ready\", \"cities\": [{\"name\": \"London\", \"lat\": 51.51, \"lon\": -0.13}, {\"name\": \"
--- ../logs/queue.json (first 600 chars) ---
[]

--- ../logs/runtime.json (first 600 chars) ---
[
  {
    "runtime_id": 222437594334494720,
    "name": "server-local",
    "component_type": "server_api",
    "base_url": "http://localhost:8082/api",
    "status": "ready",
    "labels": null,
    "capabilities": null,
    "capacity": null,
    "runti